Read the csv file and print columns/current indexing system

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gp

data = pd.read_csv("./CaliFP.csv", header=0)
print(data.columns)
print(data.index)


Index(['OBJECTID', 'YEAR_', 'STATE', 'AGENCY', 'UNIT_ID', 'FIRE_NAME',
       'INC_NUM', 'ALARM_DATE', 'CONT_DATE', 'CAUSE', 'COMMENTS', 'REPORT_AC',
       'GIS_ACRES', 'C_METHOD', 'OBJECTIVE', 'FIRE_NUM', 'COMPLEX_NAME',
       'COMPLEX_INCNUM', 'SHAPE_Length', 'SHAPE_Area'],
      dtype='object')
RangeIndex(start=0, stop=21688, step=1)


Remove fires not in california and before 1950

In [198]:
data.set_index('STATE' , inplace = True) #run entire cell only once after reading from above cell

print('entries:', len(data))
data.drop(index = 'NV' , axis = 0, inplace = True) 
data.drop(index = 'OR' , axis = 0, inplace = True) 
data.drop(index = 'AZ' , axis = 0 , inplace = True)
print('entries:', len(data))

# data = data[data['STATE'] != 'AZ'] alternate method of cleaning

data.reset_index('STATE' , inplace = True)

data.set_index('YEAR_' , inplace = True)

# F_before_1950 = data[data.index < 1950]
# print(F_before_1950)
for year in range(1898,1950): #range of known years provided in csv we need removed
    if (year in data.index):
        data.drop(index = year , axis = 0 , inplace = True)
print('entries:', len(data))
data.reset_index('YEAR_' , inplace = True)

entries: 21688
entries: 21593
entries: 16449


delete entries with missing year

In [199]:
print(len(data))
print(data['YEAR_'].isna().sum()) #isna() is bool but python is 'ducklike' and can add with .sum()
data = data[data['YEAR_'].notna()] #reassigns dataframe to the same datafram with column year with no missing year entries
print(len(data))

16449
77
16372


sort dataframe chronologically

https://en.wikipedia.org/wiki/List_of_California_wildfires
https://frap.fire.ca.gov/mapping/gis-data/
URLS to investigate tomorrow, also look into python libraries for working with GIS data

In [200]:
data['datetime'] = pd.to_datetime(data['ALARM_DATE']) #create new datetime column
data.set_index('datetime' , inplace = True) #set new column to index
data.sort_values(by='datetime', ascending = True, inplace = True) #sort values
data.reset_index('datetime' , inplace = True) #reset index to numbered list
data.drop('datetime' , axis = 1 , inplace = True) #delete the datetime column

print(len(data) , data.index)

for i in range(8):
    print(data[data.index == i*1000].ALARM_DATE)

16372 RangeIndex(start=0, stop=16372, step=1)
0    1950/05/11 00:00:00+00
Name: ALARM_DATE, dtype: object
1000    1956/10/16 00:00:00+00
Name: ALARM_DATE, dtype: object
2000    1966/05/01 00:00:00+00
Name: ALARM_DATE, dtype: object
3000    1974/05/22 00:00:00+00
Name: ALARM_DATE, dtype: object
4000    1980/06/19 00:00:00+00
Name: ALARM_DATE, dtype: object
5000    1985/07/03 00:00:00+00
Name: ALARM_DATE, dtype: object
6000    1990/08/30 00:00:00+00
Name: ALARM_DATE, dtype: object
7000    1996/08/27 00:00:00+00
Name: ALARM_DATE, dtype: object


look up fires based on location 